In [3]:
import pyarrow.feather as feather
import boto3
import pandas as pd
from sagemaker import get_execution_role


read_df = pd.read_feather("s3://mt5599/dengue_cases/dataset.feather")
read_df

,level_0,index,id,title,zoom_lat,zoom_lon,zoom_level,alert_id,feed_id,summary,...,admin1_code,admin2_code,admin3_code,admin4_code,cases_tags,deaths_tags,dates_end,precipitation_value,precipitation_mean,issue_month
0,0,0,8163513,['Dengue/DHF update (01): Americas'],39.783730,-100.445882,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,...,None,None,None,None,None,None,NaT,0.016949,0.070039,2
1,1,1,8163513,['Dengue/DHF update (01): Americas'],16.825979,-88.760093,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,...,None,None,None,None,None,None,NaT,0.116711,0.248139,2
2,2,2,8163513,['Dengue/DHF update (01): Americas'],10.273563,-84.073910,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,...,None,None,None,None,None,None,NaT,0.157389,0.424978,2
3,3,3,8163513,['Dengue/DHF update (01): Americas'],13.800038,-88.914068,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,...,None,None,None,None,None,None,NaT,0.019579,0.198830,2
4,4,4,8163513,['Dengue/DHF update (01): Americas'],15.635609,-89.898809,5,8163513,1,PRO/EDR> Dengue/DHF update (01): Americas,...,None,None,None,None,None,None,NaT,0.187086,0.292394,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,2107,2107,17428,['Zika virus - Micronesia: Yap (04)'],6.492380,159.404221,5,17428,1,PRO/EDR> Zika virus - Micronesia: Yap (04),...,00,None,None,None,[case],None,2007-07-14,0.437677,0.409450,7
2099,2108,2108,15686,['Zika virus outbreak - Micronesia (Yap) (02):...,6.492380,159.404221,5,15686,1,PRO/AH/EDR> Zika virus outbreak - Micronesia (...,...,00,None,None,None,"[case, confirmed]",None,2007-07-03,0.437677,0.409450,7
2100,2109,2109,15805,"['Zika virus - Micronesia: Guam ex Yap, susp.']",6.492380,159.404221,5,15805,1,PRO/AH/EDR> Zika virus - Micronesia: Guam ex Y...,...,00,None,None,None,[case],None,2007-07-04,0.437677,0.409450,7
2101,2110,2110,16416,['Zika virus - Micronesia: Yap (03)'],6.492380,159.404221,5,16416,1,PRO/EDR> Zika virus - Micronesia: Yap (03),...,00,None,None,None,[case],None,2007-07-07,0.437677,0.409450,7


In [2]:
import tqdm
import pandas as pd

weeks_list = {}

for i in tqdm.tqdm(range(1,54)):
    name = "week" + str(i)
    weeks_list[name] = pd.read_csv("s3://mt5599/dengue_cases/" + name + ".csv")
    weeks_list[name] = weeks_list[name].set_index(["Country or Subregion", "Year"])
    weeks_list[name] = weeks_list[name].pivot_table(index=["Country or Subregion", "Year"], columns='Measure Names', values="Measure Values")
    weeks_list[name]['Epi. Week (a)'] = weeks_list[name]['Epi. Week (a)'].fillna(int(i))

100%|██████████| 53/53 [00:04<00:00, 11.75it/s]


In [16]:
weeks = pd.concat(weeks_list)
weeks = weeks.reset_index().set_index(["Country or Subregion", "Year"])
weeks = weeks.sort_values(by=["Country or Subregion", "Year", 'Epi. Week (a)'])
weeks = weeks.set_index(['Epi. Week (a)'], append=True)
weeks

Measure Names                           level_0  % Lab Conf (x100)  \
Country or Subregion Year Epi. Week (a)                              
Andean Subregion     2014 1.0             week1          10.104352   
                          2.0             week2          15.407565   
                          3.0             week3          17.533373   
                          4.0             week4          15.194062   
                          5.0             week5          21.347135   
...                                         ...                ...   
Virgin Islands (US)  2022 38.0           week49                NaN   
                          38.0           week50                NaN   
                          38.0           week51                NaN   
                          38.0           week52                NaN   
                          38.0           week53                NaN   

Measure Names                            (SD/D) x100 (e)   CFR (f)  Deaths  \
Country or Subregion Year Epi. Week (a)                                      
Andean Subregion     2014 1.0                   1.883431  0.000000     0.0   
                          2.0                   1.608950  0.000000     0.0   
                          3.0                   1.534170  0.000000     0.0   
                          4.0                   1.498605  0.004727     1.0   
                          5.0                   1.447645  0.007501     2.0   
...                                                  ...       ...     ...   
Virgin Islands (US)  2022 38.0                       NaN       NaN     0.0   
                          38.0                       NaN       NaN     0.0   
                          38.0                       NaN       NaN     0.0   
                          38.0                       NaN       NaN     0.0   
                          38.0                       NaN       NaN     0.0   

Measure Names                            Incidence Rate (c)  \
Country or Subregion Year Epi. Week (a)                       
Andean Subregion     2014 1.0                      2.859930   
                          2.0                      6.831367   
                          3.0                     10.960031   
                          4.0                     15.397326   
                          5.0                     19.408797   
...                                                     ...   
Virgin Islands (US)  2022 38.0                     0.000000   
                          38.0                     0.000000   
                          38.0                     0.000000   
                          38.0                     0.000000   
                          38.0                     0.000000   

Measure Names                            Laboratory Confirmed  \
Country or Subregion Year Epi. Week (a)                         
Andean Subregion     2014 1.0                           397.0   
                          2.0                          1446.0   
                          3.0                          2640.0   
                          4.0                          3214.0   
                          5.0                          5692.0   
...                                                       ...   
Virgin Islands (US)  2022 38.0                            0.0   
                          38.0                            0.0   
                          38.0                            0.0   
                          38.0                            0.0   
                          38.0                            0.0   

Measure Names                            Population X 1000  Severe Dengue (d)  \
Country or Subregion Year Epi. Week (a)                                         
Andean Subregion     2014 1.0                     137381.0               74.0   
                          2.0                     137381.0              151.0   
                          3.0                     137381.0              231.0   
   

In [17]:
weeks.reset_index(inplace=True)
weeks.reset_index(inplace=True, drop=True)
#weeks.drop(["level_0"], axis=1, inplace=True)

In [23]:
weeks.rename(index={"Measure Names": "index"}, inplace=True)
#weeks.drop(["level_0"], inplace=True, axis=1)

In [22]:
pd.set_option('display.max_rows', None)

weeks[weeks["Country or Subregion"] == "Argentina"]

Measure Names,index,Country or Subregion,Year,Epi. Week (a),level_0,% Lab Conf (x100),(SD/D) x100 (e),CFR (f),Deaths,Incidence Rate (c),Laboratory Confirmed,Population X 1000,Severe Dengue (d),Total of Dengue Cases (b)
1431,1431,Argentina,2014,1.0,week1,0.000000,0.000000,0.000000,0.0,0.150707,0.0,41803.0,0.0,63.0
1432,1432,Argentina,2014,2.0,week2,0.000000,0.000000,0.000000,0.0,0.272708,0.0,41803.0,0.0,114.0
1433,1433,Argentina,2014,3.0,week3,0.000000,0.000000,0.000000,0.0,0.272708,0.0,41803.0,0.0,114.0
1434,1434,Argentina,2014,4.0,week4,0.000000,0.000000,0.000000,0.0,0.459297,0.0,41803.0,0.0,192.0
1435,1435,Argentina,2014,5.0,week5,1.577287,0.000000,0.000000,0.0,0.758319,5.0,41803.0,0.0,317.0
1436,1436,Argentina,2014,6.0,week6,1.587302,0.000000,0.000000,0.0,1.054948,7.0,41803.0,0.0,441.0
1437,1437,Argentina,2014,7.0,week7,1.838235,0.000000,0.000000,0.0,1.301342,10.0,41803.0,0.0,544.0
1438,1438,Argentina,2014,8.0,week8,2.163833,0.000000,0.000000,0.0,1.547736,14.0,41803.0,0.0,647.0
1439,1439,Argentina,2014,9.0,week9,1.931034,0.000000,0.000000,0.0,1.734325,14.0,41803.0,0.0,725.0
1440,1440,Argentina,2014,10.0,week10,1.870324,0.000000,0.000000,0.0,1.918523,15.0,41803.0,0.0,802.0


In [27]:
weeks

Measure Names,Country or Subregion,Year,Epi. Week (a),% Lab Conf (x100),(SD/D) x100 (e),CFR (f),Deaths,Incidence Rate (c),Laboratory Confirmed,Population X 1000,Severe Dengue (d),Total of Dengue Cases (b)
0,Andean Subregion,2014,1.0,10.104352,1.883431,0.000000,0.0,2.859930,397.0,137381.0,74.0,3929.0
1,Andean Subregion,2014,2.0,15.407565,1.608950,0.000000,0.0,6.831367,1446.0,137381.0,151.0,9385.0
2,Andean Subregion,2014,3.0,17.533373,1.534170,0.000000,0.0,10.960031,2640.0,137381.0,231.0,15057.0
3,Andean Subregion,2014,4.0,15.194062,1.498605,0.004727,1.0,15.397326,3214.0,137381.0,317.0,21153.0
4,Andean Subregion,2014,5.0,21.347135,1.447645,0.007501,2.0,19.408797,5692.0,137381.0,386.0,26664.0
...,...,...,...,...,...,...,...,...,...,...,...,...
28032,Virgin Islands (US),2022,38.0,NaN,NaN,NaN,0.0,0.000000,0.0,106.0,0.0,0.0
28033,Virgin Islands (US),2022,38.0,NaN,NaN,NaN,0.0,0.000000,0.0,106.0,0.0,0.0
28034,Virgin Islands (US),2022,38.0,NaN,NaN,NaN,0.0,0.000000,0.0,106.0,0.0,0.0
28035,Virgin Islands (US),2022,38.0,NaN,NaN,NaN,0.0,0.000000,0.0,106.0,0.0,0.0


In [24]:
!pip install feather-format

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached feather_format-0.4.1-py3-none-any.whl


In [ ]:
weeks

In [28]:
import feather

weeks.to_feather("../data/PAHO_dengue_by_epi_week.feather")

In [30]:
import boto3

boto3.Session().resource('s3').Bucket("mt5599").Object("dengue_cases/PAHO_dengue_by_epi_week.feather").upload_file("../data/PAHO_dengue_by_epi_week.feather")

In [12]:
week1 = pd.read_csv("s3://mt5599/dengue_cases/week1.csv")
week1

,ID,Country or Subregion,Serotype,Year,Measure Names,In / Out of Subregions,Measure Values
0,1,North America,,2022,Population X 1000,In,332915.0
1,1,North America,,2021,Population X 1000,In,332915.0
2,1,North America,,2020,Population X 1000,In,326767.0
3,1,North America,,2019,Population X 1000,In,326767.0
4,1,North America,,2018,Population X 1000,In,326767.0
...,...,...,...,...,...,...,...
5285,7,The Americas,,2018,Epi. Week (a),In,NaN
5286,7,The Americas,,2017,Epi. Week (a),In,NaN
5287,7,The Americas,,2016,Epi. Week (a),In,NaN
5288,7,The Americas,,2015,Epi. Week (a),In,NaN


In [3]:
read_df.columns

Index(['level_0', 'index', 'id', 'title', 'zoom_lat', 'zoom_lon', 'zoom_level',
       'alert_id', 'feed_id', 'summary', 'issue_date', 'load_date',
       'incident_date', 'descr', 'alert_tag_id', 'dup_count', 'dup_of',
       'unique_string', 'info_hash', 'submitted_by', 'reviewed',
       'search_string_id', 'content', 'disease', 'gpm_lat_idx', 'gpm_lon_idx',
       'precipitation_anomaly', 'location_name', 'dates_start', 'serotype',
       'total_cases', 'cases', 'severe_cases', 'deaths', 'admin1_code',
       'admin2_code', 'admin3_code', 'admin4_code', 'cases_tags',
       'deaths_tags', 'dates_end', 'precipitation_value', 'precipitation_mean',
       'issue_month'],
      dtype='object')

In [6]:
read_df[(read_df["disease"]=="dengue") & (read_df["location_name"] == "Argentina")][["dates_start","location_name","cases", "total_cases", "severe_cases", "deaths"]]

,dates_start,location_name,cases,total_cases,severe_cases,deaths
39,2020-11-28,Argentina,48415.0,59358.0,37.0,19.0
137,2020-06-20,Argentina,56095.0,79775.0,81.0,25.0
177,2020-08-15,Argentina,60666.0,61553.0,47.0,18.0
218,2020-06-20,Argentina,56095.0,79775.0,81.0,25.0
254,2020-02-29,Argentina,0.0,1706.0,0.0,0.0
291,NaT,Argentina,0.0,1706.0,0.0,0.0
328,2020-02-15,Argentina,0.0,66.0,0.0,0.0
369,2019-12-06,Argentina,1713.0,3209.0,0.0,0.0
397,2019-07-19,Argentina,1605.0,2555.0,0.0,0.0
426,2019-04-19,Argentina,632.0,1154.0,0.0,0.0


In [7]:
import pandas as pd 

dengue_df = pd.read_csv("./promed_dengue.csv", sep='\t')
dengue_df

,Unnamed: 0,id,title,zoom_lat,zoom_lon,zoom_level,alert_id,feed_id,summary,issue_date,...,descr,alert_tag_id,dup_count,dup_of,unique_string,info_hash,submitted_by,reviewed,search_string_id,content
0,8704240,8704240,"['Dengue/DHF update (04): Americas, Asia']",75.000000,-39.990234,4,8704240,1,"PRO/AH/EDR> Dengue/DHF update (04): Americas, ...",2022-07-05 02:25:03,...,Cases by country / Week updated / Serotype / T...,NaN,0,NaN,2.022071e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (04): AMERICAS, ASIA\n******..."
1,8705446,8705446,"['Dengue/DHF update (06): Americas, Europe, As...",75.000000,-39.990234,4,8705446,1,"PRO/EDR> Dengue/DHF update (06): Americas, Eur...",2022-09-07 17:16:05,...,Cases by country / Week updated / Serotype / T...,NaN,0,NaN,2.022091e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (06): AMERICAS, EUROPE, ASIA..."
2,8703206,8703206,"['Dengue/DHF update (03): Americas, Middle Eas...",18.894449,-70.484627,5,8703206,1,"PRO/AH/EDR> Dengue/DHF update (03): Americas, ...",2022-05-13 10:19:05,...,Cases by Country / Week updated / Serotype / T...,NaN,0,NaN,2.022051e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (03): AMERICAS, MIDDLE EAST,..."
3,8699963,8699963,"['Dengue/DHF update (06): Americas, Asia, rese...",13.750000,100.516670,4,8699963,1,"PRO/AH/EDR> Dengue/DHF update (06): Americas, ...",2021-11-30 03:20:05,...,Cases by Country / Week updated / Serotype / T...,NaN,0,NaN,2.021113e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (06): AMERICAS, ASIA, RESEAR..."
4,8705055,8705055,['Dengue/DHF update (05): Americas (Cuba)'],21.621170,-79.036346,5,8705055,1,PRO/EDR> Dengue/DHF update (05): Americas (Cuba),2022-08-16 03:49:17,...,NaN,NaN,0,NaN,2.022082e+07,NaN,3419.0,NaN,NaN,DENGUE/DHF UPDATE (05): AMERICAS (CUBA)\n*****...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,2190268,2190268,['Dengue eradication campaign - Brazil (02)'],-10.773110,-53.089821,4,2190268,1,PRO> Dengue eradication campaign - Brazil (02),1997-03-03 23:50:00,...,NaN,NaN,0,NaN,1.997030e+07,a1aee7d61bd1c19e8958031e4163d93f,NaN,NaN,NaN,<PRE>\n\nDENGUE ERADICATION CAMPAIGN - BRAZIL ...
1378,2190399,2190399,['Dengue - Ecuador'],-1.427420,-78.767067,5,2190399,1,PRO/EDR> Dengue - Ecuador,1997-03-22 23:50:00,...,NaN,NaN,0,NaN,1.997032e+07,4c4d8585530cfe1213f310525362031a,NaN,NaN,NaN,<PRE>\n\nDENGUE -ECUADOR\n===============\n\nD...
1379,2188692,2188692,['PROMED-EDR: Dengue/DHF - Malaysia'],2.574600,113.191803,5,2188692,1,PROMED-EDR: Dengue/DHF - Malaysia,1996-06-07 23:50:00,...,NaN,NaN,0,NaN,1.996061e+07,4568918856dd7716e10aa6768b80337a,NaN,NaN,NaN,<PRE>\n\n\nDENGUE/DENGUE HEMORRHAGIC FEVER - M...
1380,2188177,2188177,['PROMED: Dengue eradication - Brazil'],-10.773110,-53.089821,4,2188177,1,PROMED: Dengue eradication - Brazil,1996-03-22 23:50:00,...,NaN,NaN,0,NaN,1.996032e+07,dacc5a8c18b6304ea8ad74a0da8dcda2,NaN,NaN,NaN,"<PRE>\n\n\nFrom: Jack Woodall <<a href=""mailto..."


In [8]:
dengue_df['disease'] = 'dengue'

combined_df = dengue_df
combined_df

,Unnamed: 0,id,title,zoom_lat,zoom_lon,zoom_level,alert_id,feed_id,summary,issue_date,...,alert_tag_id,dup_count,dup_of,unique_string,info_hash,submitted_by,reviewed,search_string_id,content,disease
0,8704240,8704240,"['Dengue/DHF update (04): Americas, Asia']",75.000000,-39.990234,4,8704240,1,"PRO/AH/EDR> Dengue/DHF update (04): Americas, ...",2022-07-05 02:25:03,...,NaN,0,NaN,2.022071e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (04): AMERICAS, ASIA\n******...",dengue
1,8705446,8705446,"['Dengue/DHF update (06): Americas, Europe, As...",75.000000,-39.990234,4,8705446,1,"PRO/EDR> Dengue/DHF update (06): Americas, Eur...",2022-09-07 17:16:05,...,NaN,0,NaN,2.022091e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (06): AMERICAS, EUROPE, ASIA...",dengue
2,8703206,8703206,"['Dengue/DHF update (03): Americas, Middle Eas...",18.894449,-70.484627,5,8703206,1,"PRO/AH/EDR> Dengue/DHF update (03): Americas, ...",2022-05-13 10:19:05,...,NaN,0,NaN,2.022051e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (03): AMERICAS, MIDDLE EAST,...",dengue
3,8699963,8699963,"['Dengue/DHF update (06): Americas, Asia, rese...",13.750000,100.516670,4,8699963,1,"PRO/AH/EDR> Dengue/DHF update (06): Americas, ...",2021-11-30 03:20:05,...,NaN,0,NaN,2.021113e+07,NaN,3419.0,NaN,NaN,"DENGUE/DHF UPDATE (06): AMERICAS, ASIA, RESEAR...",dengue
4,8705055,8705055,['Dengue/DHF update (05): Americas (Cuba)'],21.621170,-79.036346,5,8705055,1,PRO/EDR> Dengue/DHF update (05): Americas (Cuba),2022-08-16 03:49:17,...,NaN,0,NaN,2.022082e+07,NaN,3419.0,NaN,NaN,DENGUE/DHF UPDATE (05): AMERICAS (CUBA)\n*****...,dengue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,2190268,2190268,['Dengue eradication campaign - Brazil (02)'],-10.773110,-53.089821,4,2190268,1,PRO> Dengue eradication campaign - Brazil (02),1997-03-03 23:50:00,...,NaN,0,NaN,1.997030e+07,a1aee7d61bd1c19e8958031e4163d93f,NaN,NaN,NaN,<PRE>\n\nDENGUE ERADICATION CAMPAIGN - BRAZIL ...,dengue
1378,2190399,2190399,['Dengue - Ecuador'],-1.427420,-78.767067,5,2190399,1,PRO/EDR> Dengue - Ecuador,1997-03-22 23:50:00,...,NaN,0,NaN,1.997032e+07,4c4d8585530cfe1213f310525362031a,NaN,NaN,NaN,<PRE>\n\nDENGUE -ECUADOR\n===============\n\nD...,dengue
1379,2188692,2188692,['PROMED-EDR: Dengue/DHF - Malaysia'],2.574600,113.191803,5,2188692,1,PROMED-EDR: Dengue/DHF - Malaysia,1996-06-07 23:50:00,...,NaN,0,NaN,1.996061e+07,4568918856dd7716e10aa6768b80337a,NaN,NaN,NaN,<PRE>\n\n\nDENGUE/DENGUE HEMORRHAGIC FEVER - M...,dengue
1380,2188177,2188177,['PROMED: Dengue eradication - Brazil'],-10.773110,-53.089821,4,2188177,1,PROMED: Dengue eradication - Brazil,1996-03-22 23:50:00,...,NaN,0,NaN,1.996032e+07,dacc5a8c18b6304ea8ad74a0da8dcda2,NaN,NaN,NaN,"<PRE>\n\n\nFrom: Jack Woodall <<a href=""mailto...",dengue


In [11]:
combined_df.to_feather("../data/combined_df.feather")

In [19]:
#combined_df["issue_date"] = [d.strftime("%Y-%m-%d") for d in combined_df["issue_date"]]
combined_df.loc[combined_df.issue_date.str.startswith("2020-05-27")]["content"][20]

'DENGUE/DHF UPDATE(06): ASIA, PACIFIC\n************************************\nA ProMED-mail post\n<http://www.promedmail.org>\nProMED-mail is a program of the\nInternational Society for Infectious Diseases\n<http://www.isid.org>\n\nIn this update:\r\nCases in various countries\r\nAsia\r\n---\r\nSri Lanka\r\n- National\r\nBangladesh\r\n- National\r\nIndia\r\n- Kerala\r\nPhilippines\r\n- Pangasinan\r\nMalaysia\r\n- Johor\r\nSingapore\r\n- National\r\n- National\r\nThailand\r\n- Nakhon Ratchasima\r\nPacific\r\n- Fiji & Marshall Islands\r\n- Reunion Island\r\n\r\n******\r\nCases in various countries\r\nAsia\r\n---\r\nSri Lanka\r\n- National. 13 May 2020. Over 18 900 dengue patients have been reported in Sri Lanka so far this year [2020], as the National Dengue Eradication Unit warned that cases may further rise following the monsoon season.\r\nAs of [12 May 2020] a total of 18 977 dengue patients were reported in the country for the year 2020, according to data obtained from the Epidemiolog

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
from tqdm import tqdm
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim
from datetime import datetime

#from epitator.geoname_annotator import GeonameAnnotator
#from epitator.date_annotator import DateAnnotator
#from epitator.count_annotator import CountAnnotator
#from epitator.annotator import AnnoDoc

import re
import sys
import spacy
import os
os.environ['SPACY_MODEL_SHORTCUT_LINK'] = 'en_core_web_trf'

spacy.require_gpu()

sys.path.append('../EpiTator')


locator = Nominatim(user_agent="ppcoom")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1/20)
dengue_regex = re.compile(
    r'([A-Za-z ]+).*\[w\/e (.+)\] \/ (.+) \/ (.+) \/ (.+) \/ (.+) \/ (.+)', re.MULTILINE)

tqdm.pandas()

# setup our BART transformer summarization model
print('loading transformers')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn').cuda()


def clean(content):
    split = content.splitlines()
    last_index = -1
    lower = [x.lower().strip() for x in split]
    if '--' in lower:
        last_index = lower.index('--')
    elif 'communicated by:' in lower:
        last_index = lower.index('communicated by:')-1

    cleaned = split[12:last_index]
    return '\n'.join([x for x in cleaned if x])

# helper function to summarize an input text with the BART model


def summarizer(text: str) -> str:
    input_ids = tokenizer(text, return_tensors='pt', max_length=1024,
                          padding=True, truncation=True)['input_ids']
    summary_ids = model.generate(input_ids.cuda())
    summary = ''.join([tokenizer.decode(s) for s in summary_ids])
    summary = summary.replace('<s>', '').replace('</s>', '')
    return summary

# function that extracts location names/admin codes/lat/lng, case and death counts, and date ranges from the input string
# uses epitator since it already trained rules for extracting medical/infectious disease data

def parse_dengue(row):
    return pd.DataFrame([(
        *row[2:],
        match[0].strip(),
        pd.NA if re.match(r'\d+ \w{3} \w{4}', re.sub(r'[^\w ]+', '', match[1])) is None
        else datetime.strptime(re.sub(r'[^\w ]+', '', match[1]), r'%d %b %Y'),
        match[2],
        *[pd.NA if not match[i].replace(' ', '').isnumeric() else int(match[i].replace(' ', '')) for i in range(3, 7)])
        for match in dengue_regex.findall(row['content'])],
        columns=[
        *row[2:].keys(),
        'location_name',
        'dates_start',
        'serotype',
        'total_cases',
        'confirmed_cases',
        'severe_cases',
        'deaths'
    ])



print('Opening df')
df = pd.read_feather('../data/combined_df.feather')

print('Cleaning')
df['content'] = df['content'].progress_apply(clean)
df = df[df['content'].str.contains(
    '|'.join(('case', 'cases', 'death', 'deaths')))]

dengue_df = df[df['disease'] == 'dengue']

print('Parsing dengue')
dengue_df = pd.concat([parse_dengue(row)
                      for _, row in tqdm(dengue_df.iterrows())])
dengue_df['location'] = dengue_df['location_name'].progress_apply(geocode)
dengue_df['point'] = dengue_df['location'].progress_apply(
    lambda loc: tuple(loc.point) if loc else None)
dengue_df[['location_lat', 'location_lon', 'altitude']] = pd.DataFrame(
    dengue_df['point'].tolist(), index=dengue_df.index)
dengue_df = dengue_df.drop(['location', 'point', 'altitude'], axis=1)
dengue_df = dengue_df.rename({'confirmed_cases', 'cases'})
dengue_df = dengue_df.reset_index(drop=True)

print('Finishing up')
full_df = dengue_df#pd.concat([df, dengue_df], axis=0, ignore_index=True)
full_df.to_feather('dataset.feather')
full_df = full_df.drop(['Unnamed: 0', 'index'], axis=1)
full_df.to_feather('dataset.feather')

In [9]:
!sudo apt-get install libpython3.5-dev
!pip3 install regex

/bin/bash: sudo: command not found
Keyring is skipped due to an exception: 'keyring.backends'


In [10]:
!pip install epitator

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached EpiTator-1.3.5-py3-none-any.whl
  Using cached pyparsing-2.2.0-py2.py3-none-any.whl (56 kB)
  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached spacy-2.1.8-cp37-cp37m-manylinux1_x86_64.whl (30.8 MB)
  Using cached regex-2018.01.10.tar.gz (612 kB)
  Preparing metadata (setup.py) ... done
  Using cached rdflib-6.2.0-py3-none-any.whl (500 kB)
  Using cached dateparser-0.7.1-py2.py3-none-any.whl (351 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached blis-0.2.4-cp37-cp37m-manylinux1_x86_64.whl (3.2 MB)
  Using cached plac-0.9.6-py2.py3-none-any.whl (20 kB)
  Using cached srsly-1.0.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (208 kB)
  Using cached preshed-2.0.1-cp37-cp37m-manylinux1_x86_64.whl (82 kB)
  Using cached thinc-7.0.8-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
  Using cached isodate-0.6.1-py2.py3-non

In [3]:
!pip install spacy

Keyring is skipped due to an exception: 'keyring.backends'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 38.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.4/814.4 kB 10.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.4 MB/s eta 0:00:00:00:01
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached cymem-2.0.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36 kB)
  Using cached murmurhash-1.0.9-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.3 MB/s eta 0:00:00ta 0:00:01
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached preshed-3.0.8-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (126 kB)
  Using cached srsly-2.4.5-cp37-cp37m-manylinux_2_17_

In [6]:
!pip install regex

Keyring is skipped due to an exception: 'keyring.backends'
